In [3]:
# Data
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

# Model
import torch.nn as nn
import torch.nn.functional as F
import torch

from torchvision import models

# Optim
import torch.optim as optim
from torch.autograd import Variable

# Plot graphs
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb

### Installing the libraries

In [4]:
# Data
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

# Model
import torch.nn as nn
import torch.nn.functional as F
import torch

from torchvision import models

# Optim
import torch.optim as optim
from torch.autograd import Variable

# Plot graphs
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb

### Key variables

In [6]:
wandb.init(project="emotion_recognition")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
Traceback (most recent call last):
  File "/Users/tanmaydeshmukh/miniconda3/envs/deep-learning/lib/python3.11/site-packages/wandb/sdk/wandb_init.py", line 1162, in init
    wi.setup(kwargs)
  File "/Users/tanmaydeshmukh/miniconda3/envs/deep-learning/lib/python3.11/site-packages/wandb/sdk/wandb_init.py", line 306, in setup
    wandb_login._login(
  File "/Users/tanmaydeshmukh/miniconda3/envs/deep-learning/lib/python3.11/site-packages/wandb/sdk/wandb_login.py", line 298, in _login
    wlogin.prompt_api_key()
  File "/Users/tanmaydeshmukh/miniconda3/envs/deep-learning/lib/python3.11/site-packages/wandb/sdk/wandb_login.py", line 221, in prompt_api_key
    key, status = self._prompt_api_key()
                  ^^^^^^^^^^^^^^^^^

Error: An unexpected error occurred

In [10]:
!ls ./archive/train/

angry    disgust  fear     happy    neutral  sad      surprise


In [27]:
!ls ./archive/test/surprise | wc -l

     831


In [19]:
3995 + 436 + 4097 + 7215 + 4965 + 4830 + 3171

28709

In [28]:
958 + 111 + 1024 + 1774 +  1233 + 1247 + 831

7178

In [29]:
# Set Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the folders with train, test and validation datasets (in "/My Drive/..." or from your local repository where you have downloaded data):
train = 'train'
test = 'test'

# The shape of the RGB image
img_width, img_height, channels = 48, 48, 1 # you can try different sizes

# input shape
input_shape = (img_width, img_height, 3)
# position matters!
# Number_of_channels can be at the first or the last position
# in our case - "channels last"

# minibatch size
batch_size = 128
# train set sizex
nb_train_samples = 28709
# validation set size
#nb_validation_samples = 
# test set size
nb_test_samples = 7178

# Set Epochs
epochs = 10

# Set patience for early stopping
patience = 5

In [36]:
train_transforms = A.Compose([
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.1, brightness_limit=0.1,contrast_limit=0.1),
                            A.RandomGamma(gamma_limit=(90, 110), p=0.2),
                            A.Rotate(limit = 5, p=0.9),
                            A.RGBShift(p=0.2),
                            A.GaussNoise(p=0.1),
                            A.ChannelShuffle(p=0.2),
                            A.ToGray(p=0.1),
                            A.MedianBlur(p=0.1),
                            A.CLAHE(p=0.1),
                            A.pytorch.ToTensorV2()
                    ])

test_transforms = A.pytorch.ToTensorV2()

In [37]:
train_path = './archive/train/'
test_path = './archive/test/'

In [38]:
train_data = ImageFolder(train_path, transform=train_transforms)
test_data = ImageFolder(test_path, transform=test_transforms)

In [39]:
BATCH_SIZE = batch_size

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

In [40]:
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Length of train dataloader: 225 batches of 128
Length of test dataloader: 57 batches of 128


In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()

    def forward(self, x):
        return x